In [67]:
a = [1,2,3,4,5,6,7,8,9,10,11,12]
b = [2022,2021,2020]
c = 1

for i in b:
    for j in a:
        start_date = datetime.date(i, j, c)
        end_date = datetime.date(i,j,calendar.monthrange(i,j)[1])
        start_date = str(start_date)
        end_date = str(end_date)
        query = ("insert into hive.fact.order_fact_v11 with cn_order as ( select entry_time, n_slot_id, actual_delivery_time, id, item_count, no_of_bags, coupon_code, consumer_id, agent_id, total_amount, net_amount, discount_amount, packaging_charge, star_rating, status from hive.consumer.cn_order where dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' and agent_id <> 14526 ), cn_order_rating as ( select star_rating, order_id from hive.consumer.cn_order_rating where cast(entry_time as timestamp) between cast('"+start_date_minus_two+"' as timestamp) and cast('"+end_date_plus_two+"' as timestamp) ), cn_order_move_history as ( select order_id from hive.consumer.cn_order_move_history where dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' group by 1 ), cn_order_data as ( select id, order_id, prod_id, transit_quantity, quantity, price, dt from hive.consumer.cn_order_data where dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' ), cn_inc_ledger as ( select order_id, incentive, inc_type, n_slot_id, agent_id from hive.consumer.cn_inc_ledger where inc_type in ('dl','do','mk','mg') and agent_id <> 28081 ), cn_wallet_transaction as ( select order_id, added_by, credit, debit from hive.consumer.cn_wallet_transaction where dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' and order_id > 0 ), order_marketing_cost as ( select order_id, sum(marketing_cost) as order_marketing_amount from hive.products.order_marketing_cost where order_id in (select id from cn_order) group by 1 ), cn_ticket as ( select order_id, max(case when lower(issue_type) = 'item' then true else false end) as item_issue_type_flag, max(case when lower(issue_type) = 'order' then true else false end) as order_issue_type_flag, array_distinct(array_agg(id)) as ticket_id_array, max(created) as created, max(status) as status, max(freshdesk_ticket_id) as freshdesk_ticket_id, max_by(is_auto_refund, created) as is_auto_refund_flag, max_by(total_refunded, created) as total_refunded from hive.consumer.cn_ticket where dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' group by 1 ), cr_order as ( select retailer_id, actual_delivery_time, date_format(date_add('minute', +330, from_unixtime(actual_delivery_time)),'%Y-%m-%d %H:%m:%s') as reach_time_ist, n_slot_id from hive.products.cr_order where actual_delivery_time > 0 and dt between '"+start_date_minus_two+"' and '"+end_date_plus_two+"' ), order_data_property as ( select order_data_id, subscription_quantity, discount_amount from hive.consumer.order_data_property ), order_rank as ( select consumer_id, order_id, overall_orders_rank, delivered_orders_rank from hive.data.order_rank where dt = (select max(dt) from hive.data.order_rank where dt >= cast(date_add('day',-4,current_date) as varchar)) ), joins_1 as ( select a.id as agent_id, a.zone_polygon_id, a.zone_id, c.name as zone_name, d.name as polygon_name from hive.crofarmusers.cr_retailer a left join hive.crofarmusers.cr_zone c on a.zone_id = c.id left join hive.crofarmusers.cr_zone_polygon d on a.zone_polygon_id = d.id group by 1,2,3,4,5 ), aggs_1 as ( select a.order_id, coalesce(sum(cast(quantity as double)*cast(price as double)),0) as order_value, coalesce(sum(case when product_type = 0 then (cast(quantity as double)*cast(price as double)) end),0) as fnv_order_value , coalesce(sum(case when product_type = 1 then (cast(quantity as double)*cast(price as double)) end),0) as fmcg_order_value, coalesce(sum(case when cat in ('E') then (cast(quantity as double)*cast(price as double)) end),0) as exotic_order_value, coalesce(sum(case when cat in ('F') then (cast(quantity as double)*cast(price as double)) end),0) as fruit_order_value, coalesce(sum(case when cat in ('G') then (cast(quantity as double)*cast(price as double)) end),0) as greens_order_value, coalesce(sum(case when cat in ('O','OF','B','D') then (cast(quantity as double)*cast(price as double)) end),0) as other_order_value, coalesce(sum(case when cat in ('P') then (cast(quantity as double)*cast(price as double)) end),0) as pot_order_value, coalesce(sum(case when cat in ('T') then (cast(quantity as double)*cast(price as double)) end),0) as free_order_value, coalesce(sum(cast(transit_quantity as double)*cast(price as double)),0) as order_transit_value, coalesce(sum(case when product_type = 0 then (cast(transit_quantity as double)*cast(price as double)) end),0) as fnv_order_transit_value, coalesce(sum(case when product_type = 1 then (cast(transit_quantity as double)*cast(price as double)) end),0) as fmcg_order_transit_value, coalesce(sum(case when cat in ('E') then (cast(transit_quantity as double)*cast(price as double)) end),0) as exotic_order_transit_value, coalesce(sum(case when cat in ('F') then (cast(transit_quantity as double)*cast(price as double)) end),0) as fruit_order_transit_value, coalesce(sum(case when cat in ('G') then (cast(transit_quantity as double)*cast(price as double)) end),0) as greens_order_transit_value, coalesce(sum(case when cat in ('O','OF','B','D') then (cast(transit_quantity as double)*cast(price as double)) end),0) as other_order_transit_value, coalesce(sum(case when cat in ('P') then (cast(transit_quantity as double)*cast(price as double)) end),0) as pot_order_transit_value, coalesce(sum(case when cat in ('T') then (cast(transit_quantity as double)*cast(price as double)) end),0) as free_order_transit_value, array_distinct(array_agg(case when transit_quantity <> '0.00' then prod_id end)) as order_delivered_items_list, array_distinct(array_agg(prod_id)) as order_placed_items_list, cardinality(array_distinct(array_agg(prod_id))) as order_placed_item_count, cardinality(array_distinct(array_agg(case when transit_quantity <> '0.00' then prod_id end))) as order_delivered_items_count from cn_order_data a left join hive.products.product b on a.prod_id =b.id left join mysql.temp_finance.sub1 c on a.prod_id = c.id where b.dt = (select max(dt) from hive.products.product where dt >= cast(date_add('day',-4,current_date) as varchar)) group by 1 ), aggs_2 as ( select order_id, sum(case when added_by = 500000 then cast(credit as double) else cast(0 as double) end) as weight_issue_amount, (sum(case when added_by = 100000 then cast(credit as double) else cast(0 as double) end) - sum(case when added_by = 300500 then cast(debit as double) else cast(0 as double) end)) as short_fall_amount from cn_wallet_transaction group by 1 ), aggs_3 as ( select order_id, sum(cast(incentive as double)) as incentive_amount from cn_inc_ledger group by 1 ), joins_2 as ( select a.order_id, a.id, b.order_data_id, b.subscription_quantity, b.discount_amount as subscription_discount_amount, a.prod_id, a.price from cn_order_data a left join order_data_property b on a.id = b.order_data_id ), aggs_4 as ( select order_id, sum(cast(subscription_quantity as double)) as subscription_quantity, sum(cast(subscription_discount_amount as double)) as subscription_discount_amount, array_distinct(array_agg(prod_id)) as subscription_products, cardinality(array_distinct(array_agg(prod_id))) as subscription_product_count, sum(cast(subscription_quantity as double)*cast(price as double)) as subscription_value from joins_2 where order_data_id is not null group by 1 ), joins as ( select a.id as order_id, a.status as order_status, k.overall_orders_rank, k.delivered_orders_rank, a.consumer_id, a.agent_id, b.zone_polygon_id, b.polygon_name, b.zone_id, b.zone_name, a.n_slot_id, cast(replace(a.entry_time,'T',' ') as timestamp(3)) as placed_datetime_utc, date_add('minute',+330,cast(replace(a.entry_time,'T',' ') as timestamp(3))) as placed_datetime_ist, date(cast(replace(a.entry_time,'T',' ') as timestamp(3))) as placed_date_utc, date(date_add('minute',+330,cast(replace(a.entry_time,'T',' ') as timestamp(3)))) as placed_date_ist, date_format(from_unixtime(a.actual_delivery_time),'%Y-%m-%d %H:%m:%s') as delivery_datetime_utc, date_format(date_add('minute',+330, from_unixtime(a.actual_delivery_time)),'%Y-%m-%d %H:%m:%s') as delivery_datetime_ist, date_format(from_unixtime(a.actual_delivery_time),'%Y-%m-%d') as delivery_date_utc, date_format(date_add('minute',+330, from_unixtime(a.actual_delivery_time)),'%Y-%m-%d') as delivery_date_ist, c.reach_time_ist, coalesce(a.item_count,0) as item_count, coalesce(a.no_of_bags,0) as bags, coalesce(a.total_amount,'0') as total_amount, coalesce(a.discount_amount,'0') as discount_amount, coalesce(a.net_amount,'0') as net_amount, coalesce(a.packaging_charge,0) as packaging_amount, a.coupon_code as coupon_code, coalesce(e.order_marketing_amount,0) as marketing_amount, coalesce(g.incentive_amount,0) as incentive_amount, coalesce(h.order_id,0) as order_id_move, coalesce(a.star_rating,'0') as pre_rating, coalesce(i.star_rating,'0') as post_rating, coalesce(j.order_value,0) as value, coalesce(j.fnv_order_value,0) as fnv_value, coalesce(j.fmcg_order_value,0) as fmcg_value, coalesce(j.exotic_order_value,0) as exotic_value, coalesce(j.fruit_order_value,0) as fruit_value, coalesce(j.greens_order_value,0) as greens_value, coalesce(j.other_order_value,0) as other_value, coalesce(j.pot_order_value,0) as pot_ovalue, coalesce(j.free_order_value,0) as free_value, coalesce(j.order_transit_value,0) as transit_value, coalesce(j.fnv_order_transit_value,0) as fnv_transit_value, coalesce(j.fmcg_order_transit_value,0) as fmcg_transit_value, coalesce(j.exotic_order_transit_value,0) as exotic_transit_value, coalesce(j.fruit_order_transit_value,0) as fruit_transit_value, coalesce(j.greens_order_transit_value,0) as greens_transit_value, coalesce(j.other_order_transit_value,0) as other_transit_value, coalesce(j.pot_order_transit_value,0) as pot_transit_value, coalesce(j.free_order_transit_value,0) as free_transit_value, coalesce(j.order_delivered_items_list,array[]) as delivered_items_list, coalesce(j.order_delivered_items_count,0) as delivered_items_count, coalesce(j.order_placed_items_list,array[]) as placed_items_list, coalesce(j.order_placed_item_count,0) as placed_item_count, coalesce(l.item_issue_type_flag, false) as item_issue_type_flag, coalesce(l.order_issue_type_flag, false) as order_issue_type_flag, coalesce(l.ticket_id_array, array[]) as ticket_id_array, l.created as ticket_created_at, l.status as ticket_status, l.freshdesk_ticket_id, coalesce(l.is_auto_refund_flag, false) as is_auto_refund_flag, coalesce(l.total_refunded, '0') as total_refunded, coalesce(m.order_id, 0) as subscription_order, coalesce(m.subscription_quantity,0) as subscription_quantity, coalesce(m.subscription_discount_amount,0) as subscription_discount_amount, coalesce(m.subscription_products,array[]) as subscription_products, coalesce(m.subscription_product_count,0) as subscription_product_count, coalesce(m.subscription_value,0) as subscription_value, cast(date(date_add('minute',+330,cast(replace(a.entry_time,'T',' ') as timestamp(3)))) as varchar) as dt from cn_order a left join joins_1 b on a.agent_id = b.agent_id left join cr_order c on a.agent_id = c.retailer_id and a.n_slot_id = c.n_slot_id left join aggs_2 d on a.id = d.order_id left join order_marketing_cost e on a.id = e.order_id left join aggs_3 g on a.id = g.order_id left join cn_order_move_history h on a.id = h.order_id left join cn_order_rating i on a.id = i.order_id left join aggs_1 j on a.id = j.order_id left join order_rank k on a.id = k.order_id and a.consumer_id = k.consumer_id left join cn_ticket l on a.id = l.order_id left join aggs_4 m on a.id = m.order_id ) select * from joins where dt between '"+start_date+"' and '"+end_date+"'")
        print(query)

NameError: name 'start_date_minus_two' is not defined

In [81]:
    a3=[
    "consumer.cn_paytm_transaction",
    "consumer.cn_payu_transaction",
    "consumer.cn_razorpay_transaction",
    "consumer.gokwik_transaction",
    "consumer.icici_transaction",
    "consumer.sodexo_transaction",
    "consumer.upi_transaction",
    "consumer.cn_order_data",
]

In [88]:
b3=["products.cr_order",
"products.cr_order_data",
"products.cr_purchase_order",
"products.cr_purchase_order_data",
"products.paytm_transaction",
"products.zone_product_deal",
"products.inventory_status"]

In [6]:
a[0].split(".")[1]

'cn_wallet_transaction'

In [18]:
a1=[]
b1=[]

In [1]:
import trino
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur = conn.cursor()
# # for i in range(len(a)):
# query=("select max(dt),min(dt) from hive.historical.ut1_consumer_"+a[0].split(".")[1])
# cur.execute(query)
# rows = cur.fetchall()
# a1.append(rows[0][0])
# b1.append(rows[0][1])

In [34]:
a1[0]

'2022-07-11'

In [35]:
b1[0]

'2020-02-21'

In [32]:
if(int(a1[0].split("-")[1])==12):
    

SyntaxError: expected ':' (3816767236.py, line 1)

In [28]:
b1[0].split("-")

['2020', '02', '21']

In [11]:
("insert into hive.historical.ut_consumer_"+a[0].split(".")[1]+" select * from hive.historical.ut1_consumer_"+a[0].split(".")[1])+"where dt between"+

'insert into hive.historical.ut_consumer_cn_wallet_transaction select * from hive.historical.ut1_consumer_cn_wallet_transaction'

In [91]:
import trino
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur = conn.cursor()
a = [1,2,3,4,5,6,7,8,9,10,11,12]
b = [2022,2021,2020,2019,2018,2017,2016,2015]
c = 1
for k in range(len(b3)):
    for i in b:
        for j in a:
            start_date = datetime.date(i, j, c)
            end_date = datetime.date(i,j,calendar.monthrange(i,j)[1])
            start_date=str(start_date)
            end_date=str(end_date)
            q=("insert into hive.products."+b3[k].split(".")[1]+" select * from hive.historical.ut1_products_"+b3[k].split(".")[1]+" where dt between  '"+start_date+"' and '"+end_date+"'")
            cur.execute(q)
            rows = cur.fetchall()
            print(rows)

[[11145]]
[[10259]]
[[11800]]
[[12485]]
[[13592]]
[[14090]]
[[5165]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[16813]]
[[14787]]
[[14422]]
[[13787]]
[[14102]]
[[15041]]
[[16419]]
[[17304]]
[[15054]]
[[14164]]
[[12387]]
[[11470]]
[[6098]]
[[5136]]
[[5737]]
[[7770]]
[[10643]]
[[9884]]
[[9147]]
[[10318]]
[[11805]]
[[15420]]
[[16775]]
[[18430]]
[[2054]]
[[2491]]
[[3262]]
[[3489]]
[[3609]]
[[4486]]
[[6654]]
[[7176]]
[[7787]]
[[7023]]
[[6947]]
[[7055]]
[[1041]]
[[1485]]
[[1851]]
[[1933]]
[[2063]]
[[1827]]
[[1839]]
[[2113]]
[[2820]]
[[3018]]
[[2926]]
[[2601]]
[[429]]
[[341]]
[[402]]
[[438]]
[[535]]
[[672]]
[[720]]
[[830]]
[[682]]
[[620]]
[[579]]
[[751]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[13]]
[[115]]
[[304]]
[[724619]]
[[608838]]
[[658681]]
[[668265]]
[[727164]]
[[765148]]
[[250831]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[380868]]
[[327822]]
[[384077]]
[[422949]]
[[508929]]
[[433223]]
[[464770]]
[[489175]]
[[562790]]
[[570812]]
[[564341]]
[[635029]]
[[29239]]
[[25851]]
[[41643]]
[[103197]

In [10]:
# from datetime import datetime
# from airflow import DAG
# from airflow.operators.python import PythonOperator
# from trino.dbapi import connect
# from airflow.operators.python_operator import PythonOperator
# import subprocess


    call_queries = ["insert into hive.consumer.agent_consumer_coupon select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_consumer_coupon where dt =(select max(dt) from hive.historical.full_consumer_agent_consumer_coupon where dt >= cast(current_date as varchar))", "insert into hive.consumer.agent_consumer_invite select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_consumer_invite where dt = (select max(dt) from hive.historical.full_consumer_agent_consumer_invite where dt >= cast(current_date as varchar))", "insert into hive.consumer.agent_contact_list select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_contact_list where dt = (select max(dt) from hive.historical.full_consumer_agent_contact_list where dt >= cast(current_date as varchar))", "insert into hive.consumer.agent_coupon_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_coupon_mapping where dt = (select max(dt) from hive.historical.full_consumer_agent_coupon_mapping where dt >= cast(current_date as varchar))", "insert into hive.consumer.agent_holiday select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_holiday where dt = (select max(dt) from hive.historical.full_consumer_agent_holiday where dt >= cast(current_date as varchar))", "insert into hive.consumer.agent_holiday_history select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_agent_holiday_history where dt = (select max(dt) from hive.historical.full_consumer_agent_holiday_history where dt >= cast(current_date as varchar))", "insert into hive.consumer.announcement select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_announcement where dt = (select max(dt) from hive.historical.full_consumer_announcement where dt >= cast(current_date as varchar))", "insert into hive.consumer.auth_permission select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_auth_permission where dt = (select max(dt) from hive.historical.full_consumer_auth_permission where dt >= cast(current_date as varchar))", "insert into hive.consumer.category_incentive select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_category_incentive where dt = (select max(dt) from hive.historical.full_consumer_category_incentive where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_address select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_address where dt = (select max(dt) from hive.historical.full_consumer_cn_address where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_address_change_history select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_address_change_history where dt = (select max(dt) from hive.historical.full_consumer_cn_address_change_history where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_agent_first_order_performance select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_agent_first_order_performance where dt = (select max(dt) from hive.historical.full_consumer_cn_agent_first_order_performance where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_agent_performance select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_agent_performance where dt = (select max(dt) from hive.historical.full_consumer_cn_agent_performance where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_agent_polygon select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_agent_polygon where dt = (select max(dt) from hive.historical.full_consumer_cn_agent_polygon where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_agent_route select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_agent_route where dt = (select max(dt) from hive.historical.full_consumer_cn_agent_route where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_consumer_coupon select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_consumer_coupon where dt = (select max(dt) from hive.historical.full_consumer_cn_consumer_coupon where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_consumer_property select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_consumer_property where dt = (select max(dt) from hive.historical.full_consumer_cn_consumer_property where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_consumer_wallet select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_consumer_wallet where dt = (select max(dt) from hive.historical.full_consumer_cn_consumer_wallet where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_faq select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_faq where dt = (select max(dt) from hive.historical.full_consumer_cn_faq where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_grid_search select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_grid_search where dt = (select max(dt) from hive.historical.full_consumer_cn_grid_search where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_inc_param select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_inc_param where dt = (select max(dt) from hive.historical.full_consumer_cn_inc_param where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_issue select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_issue where dt = (select max(dt) from hive.historical.full_consumer_cn_issue where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_membership_plan_type select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_membership_plan_type where dt = (select max(dt) from hive.historical.full_consumer_cn_membership_plan_type where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_payment_deals select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_payment_deals where dt = (select max(dt) from hive.historical.full_consumer_cn_payment_deals where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_picker select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_picker where dt = (select max(dt) from hive.historical.full_consumer_cn_picker where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_polygon_search select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_polygon_search where dt = (select max(dt) from hive.historical.full_consumer_cn_polygon_search where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_sodexo_source_refund select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_sodexo_source_refund where dt = (select max(dt) from hive.historical.full_consumer_cn_sodexo_source_refund where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_sodexo_transaction_history select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_sodexo_transaction_history where dt = (select max(dt) from hive.historical.full_consumer_cn_sodexo_transaction_history where dt >= cast(current_date as varchar))", "insert into hive.consumer.cn_wallet_coupon select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cn_wallet_coupon where dt = (select max(dt) from hive.historical.full_consumer_cn_wallet_coupon where dt >= cast(current_date as varchar))", "insert into hive.consumer.consumer_coupon select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_consumer_coupon where dt = (select max(dt) from hive.historical.full_consumer_consumer_coupon where dt >= cast(current_date as varchar))", "insert into hive.consumer.consumer_retention_campaign select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_consumer_retention_campaign where dt = (select max(dt) from hive.historical.full_consumer_consumer_retention_campaign where dt >= cast(current_date as varchar))", "insert into hive.consumer.cr_firebase_user_map select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_cr_firebase_user_map where dt = (select max(dt) from hive.historical.full_consumer_cr_firebase_user_map where dt >= cast(current_date as varchar))", "insert into hive.consumer.django_content_type select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_django_content_type where dt = (select max(dt) from hive.historical.full_consumer_django_content_type where dt >= cast(current_date as varchar))", "insert into hive.consumer.django_migrations select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_django_migrations where dt = (select max(dt) from hive.historical.full_consumer_django_migrations where dt >= cast(current_date as varchar))", "insert into hive.consumer.order_data_property select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_order_data_property where dt = (select max(dt) from hive.historical.full_consumer_order_data_property where dt >= cast(current_date as varchar))", "insert into hive.consumer.order_delivery_info select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_order_delivery_info where dt = (select max(dt) from hive.historical.full_consumer_order_delivery_info where dt >= cast(current_date as varchar))", "insert into hive.consumer.otipy_metrics select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_otipy_metrics where dt = (select max(dt) from hive.historical.full_consumer_otipy_metrics where dt >= cast(current_date as varchar))", "insert into hive.consumer.partner_rejected_order select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_partner_rejected_order where dt = (select max(dt) from hive.historical.full_consumer_partner_rejected_order where dt >= cast(current_date as varchar))", "insert into hive.consumer.partner_target select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_partner_target where dt = (select max(dt) from hive.historical.full_consumer_partner_target where dt >= cast(current_date as varchar))", "insert into hive.consumer.partner_target_level select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_partner_target_level where dt = (select max(dt) from hive.historical.full_consumer_partner_target_level where dt >= cast(current_date as varchar))", "insert into hive.consumer.product_rating select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_product_rating where dt = (select max(dt) from hive.historical.full_consumer_product_rating where dt >= cast(current_date as varchar))", "insert into hive.consumer.return_order_data select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_return_order_data where dt = (select max(dt) from hive.historical.full_consumer_return_order_data where dt >= cast(current_date as varchar))", "insert into hive.consumer.route_line select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_route_line where dt = (select max(dt) from hive.historical.full_consumer_route_line where dt >= cast(current_date as varchar))", "insert into hive.consumer.sms_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_consumer_sms_mapping where dt = (select max(dt) from hive.historical.full_consumer_sms_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.auth_permission select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_auth_permission where dt = (select max(dt) from hive.historical.full_crofarmusers_auth_permission where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_address select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_address where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_address where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_ba_distributor_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_ba_distributor_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_ba_distributor_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_ba_inc_map select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_ba_inc_map where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_ba_inc_map where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_ba_manager_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_ba_manager_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_ba_manager_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_babu select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_babu where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_babu where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_babu_chain_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_babu_chain_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_babu_chain_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_babu_retailer_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_babu_retailer_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_babu_retailer_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_black_zone_polygon select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_black_zone_polygon where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_black_zone_polygon where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_business_associate select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_business_associate where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_business_associate where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_chain select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_chain where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_chain where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_city select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_city where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_city where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_city_delivery_slot select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_city_delivery_slot where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_city_delivery_slot where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_config select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_config where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_config where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_dept select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_dept where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_dept where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_distributor select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_distributor where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_distributor where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_distributor_retailer_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_distributor_retailer_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_distributor_retailer_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_distributor_supplier_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_distributor_supplier_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_distributor_supplier_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_driver select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_driver where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_driver where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_due_date_history select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_due_date_history where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_due_date_history where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_farmer select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_farmer where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_farmer where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_locality select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_locality where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_locality where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_locality_group select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_locality_group where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_locality_group where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_mandi select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_mandi where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_mandi where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_mandi_location_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_mandi_location_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_mandi_location_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_multi_user select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_multi_user where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_multi_user where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_onesig_user_map select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_onesig_user_map where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_onesig_user_map where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_partner_bank_info select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_partner_bank_info where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_partner_bank_info where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_permission select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_permission where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_permission where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_retailer select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_retailer where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_retailer where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_retailer_veg_category_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_retailer_veg_category_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_retailer_veg_category_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_role select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_role where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_role where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_role_permission_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_role_permission_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_role_permission_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_state select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_state where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_state where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_supplier select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_supplier where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_supplier where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_trans_dc_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_trans_dc_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_trans_dc_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_trans_veh_type_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_trans_veh_type_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_trans_veh_type_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_transporter select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_transporter where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_transporter where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_trip select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_trip where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_trip where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_user_location_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_user_location_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_user_location_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_user_mandi_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_user_mandi_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_user_mandi_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_veg_category select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_veg_category where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_veg_category where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_vehicle select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_vehicle where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_vehicle where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_vehicle_type select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_vehicle_type where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_vehicle_type where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_zone select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_zone where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_zone where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.cr_zone_polygon select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_cr_zone_polygon where dt = (select max(dt) from hive.historical.full_crofarmusers_cr_zone_polygon where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.django_content_type select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_django_content_type where dt = (select max(dt) from hive.historical.full_crofarmusers_django_content_type where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.django_migrations select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_django_migrations where dt = (select max(dt) from hive.historical.full_crofarmusers_django_migrations where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.partner_ba_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_partner_ba_mapping where dt = (select max(dt) from hive.historical.full_crofarmusers_partner_ba_mapping where dt >= cast(current_date as varchar))", "insert into hive.crofarmusers.retailer_document select *, cast('done' as varchar) as data_copied from hive.historical.full_crofarmusers_retailer_document where dt = (select max(dt) from hive.historical.full_crofarmusers_retailer_document where dt >= cast(current_date as varchar))", "insert into hive.products.Category select *, cast('done' as varchar) as data_copied from hive.historical.full_products_Category where dt = (select max(dt) from hive.historical.full_products_Category where dt >= cast(current_date as varchar))", "insert into hive.products.Commodity select *, cast('done' as varchar) as data_copied from hive.historical.full_products_Commodity where dt = (select max(dt) from hive.historical.full_products_Commodity where dt >= cast(current_date as varchar))", "insert into hive.products.ProdAttrbMapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_ProdAttrbMapping where dt = (select max(dt) from hive.historical.full_products_ProdAttrbMapping where dt >= cast(current_date as varchar))", "insert into hive.products.Product select *, cast('done' as varchar) as data_copied from hive.historical.full_products_Product where dt = (select max(dt) from hive.historical.full_products_Product where dt >= cast(current_date as varchar))", "insert into hive.products.SubCategory select *, cast('done' as varchar) as data_copied from hive.historical.full_products_SubCategory where dt = (select max(dt) from hive.historical.full_products_SubCategory where dt >= cast(current_date as varchar))", "insert into hive.products.agent_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_agent_product_mapping where dt = (select max(dt) from hive.historical.full_products_agent_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.auth_permission select *, cast('done' as varchar) as data_copied from hive.historical.full_products_auth_permission where dt = (select max(dt) from hive.historical.full_products_auth_permission where dt >= cast(current_date as varchar))", "insert into hive.products.ba_airtelpayment select *, cast('done' as varchar) as data_copied from hive.historical.full_products_ba_airtelpayment where dt = (select max(dt) from hive.historical.full_products_ba_airtelpayment where dt >= cast(current_date as varchar))", "insert into hive.products.ba_dp_transactions select *, cast('done' as varchar) as data_copied from hive.historical.full_products_ba_dp_transactions where dt = (select max(dt) from hive.historical.full_products_ba_dp_transactions where dt >= cast(current_date as varchar))", "insert into hive.products.bulk_order_cancel select *, cast('done' as varchar) as data_copied from hive.historical.full_products_bulk_order_cancel where dt = (select max(dt) from hive.historical.full_products_bulk_order_cancel where dt >= cast(current_date as varchar))", "insert into hive.products.bunch_packing_qc_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_bunch_packing_qc_mapping where dt = (select max(dt) from hive.historical.full_products_bunch_packing_qc_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.bunch_table_packing select *, cast('done' as varchar) as data_copied from hive.historical.full_products_bunch_table_packing where dt = (select max(dt) from hive.historical.full_products_bunch_table_packing where dt >= cast(current_date as varchar))", "insert into hive.products.category_properties select *, cast('done' as varchar) as data_copied from hive.historical.full_products_category_properties where dt = (select max(dt) from hive.historical.full_products_category_properties where dt >= cast(current_date as varchar))", "insert into hive.products.city_group select *, cast('done' as varchar) as data_copied from hive.historical.full_products_city_group where dt = (select max(dt) from hive.historical.full_products_city_group where dt >= cast(current_date as varchar))", "insert into hive.products.combo_products select *, cast('done' as varchar) as data_copied from hive.historical.full_products_combo_products where dt = (select max(dt) from hive.historical.full_products_combo_products where dt >= cast(current_date as varchar))", "insert into hive.products.cr_credit_note select *, cast('done' as varchar) as data_copied from hive.historical.full_products_cr_credit_note where dt = (select max(dt) from hive.historical.full_products_cr_credit_note where dt >= cast(current_date as varchar))", "insert into hive.products.cr_mandi_latest_price select *, cast('done' as varchar) as data_copied from hive.historical.full_products_cr_mandi_latest_price where dt = (select max(dt) from hive.historical.full_products_cr_mandi_latest_price where dt >= cast(current_date as varchar))", "insert into hive.products.cr_mandi_price select *, cast('done' as varchar) as data_copied from hive.historical.full_products_cr_mandi_price where dt = (select max(dt) from hive.historical.full_products_cr_mandi_price where dt >= cast(current_date as varchar))", "insert into hive.products.crate select *, cast('done' as varchar) as data_copied from hive.historical.full_products_crate where dt = (select max(dt) from hive.historical.full_products_crate where dt >= cast(current_date as varchar))", "insert into hive.products.crate_transaction select *, cast('done' as varchar) as data_copied from hive.historical.full_products_crate_transaction where dt = (select max(dt) from hive.historical.full_products_crate_transaction where dt >= cast(current_date as varchar))", "insert into hive.products.crate_user_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_crate_user_mapping where dt = (select max(dt) from hive.historical.full_products_crate_user_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.dc_inventory select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dc_inventory where dt = (select max(dt) from hive.historical.full_products_dc_inventory where dt >= cast(current_date as varchar))", "insert into hive.products.dc_metric select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dc_metric where dt = (select max(dt) from hive.historical.full_products_dc_metric where dt >= cast(current_date as varchar))", "insert into hive.products.deal_share select *, cast('done' as varchar) as data_copied from hive.historical.full_products_deal_share where dt = (select max(dt) from hive.historical.full_products_deal_share where dt >= cast(current_date as varchar))", "insert into hive.products.deal_share_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_deal_share_product_mapping where dt = (select max(dt) from hive.historical.full_products_deal_share_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.delivery_feedback_ticket select *, cast('done' as varchar) as data_copied from hive.historical.full_products_delivery_feedback_ticket where dt = (select max(dt) from hive.historical.full_products_delivery_feedback_ticket where dt >= cast(current_date as varchar))", "insert into hive.products.dist_retail_margin select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dist_retail_margin where dt = (select max(dt) from hive.historical.full_products_dist_retail_margin where dt >= cast(current_date as varchar))", "insert into hive.products.dist_vol_pricing select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dist_vol_pricing where dt = (select max(dt) from hive.historical.full_products_dist_vol_pricing where dt >= cast(current_date as varchar))", "insert into hive.products.distributor_machine select *, cast('done' as varchar) as data_copied from hive.historical.full_products_distributor_machine where dt = (select max(dt) from hive.historical.full_products_distributor_machine where dt >= cast(current_date as varchar))", "insert into hive.products.distributor_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_distributor_product_mapping where dt = (select max(dt) from hive.historical.full_products_distributor_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.distributor_table select *, cast('done' as varchar) as data_copied from hive.historical.full_products_distributor_table where dt = (select max(dt) from hive.historical.full_products_distributor_table where dt >= cast(current_date as varchar))", "insert into hive.products.distributor_usp select *, cast('done' as varchar) as data_copied from hive.historical.full_products_distributor_usp where dt = (select max(dt) from hive.historical.full_products_distributor_usp where dt >= cast(current_date as varchar))", "insert into hive.products.django_content_type select *, cast('done' as varchar) as data_copied from hive.historical.full_products_django_content_type where dt = (select max(dt) from hive.historical.full_products_django_content_type where dt >= cast(current_date as varchar))", "insert into hive.products.django_migrations select *, cast('done' as varchar) as data_copied from hive.historical.full_products_django_migrations where dt = (select max(dt) from hive.historical.full_products_django_migrations where dt >= cast(current_date as varchar))", "insert into hive.products.driver_bag_colloection select *, cast('done' as varchar) as data_copied from hive.historical.full_products_driver_bag_colloection where dt = (select max(dt) from hive.historical.full_products_driver_bag_colloection where dt >= cast(current_date as varchar))", "insert into hive.products.driver_location select *, cast('done' as varchar) as data_copied from hive.historical.full_products_driver_location where dt = (select max(dt) from hive.historical.full_products_driver_location where dt >= cast(current_date as varchar))", "insert into hive.products.driver_transaction select *, cast('done' as varchar) as data_copied from hive.historical.full_products_driver_transaction where dt = (select max(dt) from hive.historical.full_products_driver_transaction where dt >= cast(current_date as varchar))", "insert into hive.products.dt_retailer_location_history select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dt_retailer_location_history where dt = (select max(dt) from hive.historical.full_products_dt_retailer_location_history where dt >= cast(current_date as varchar))", "insert into hive.products.dt_route_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_dt_route_mapping where dt = (select max(dt) from hive.historical.full_products_dt_route_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.group_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_group_product_mapping where dt = (select max(dt) from hive.historical.full_products_group_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.inventory_trans_ledger select *, cast('done' as varchar) as data_copied from hive.historical.full_products_inventory_trans_ledger where dt = (select max(dt) from hive.historical.full_products_inventory_trans_ledger where dt >= cast(current_date as varchar))", "insert into hive.products.jmart_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_jmart_mapping where dt = (select max(dt) from hive.historical.full_products_jmart_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.linked_products_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_linked_products_mapping where dt = (select max(dt) from hive.historical.full_products_linked_products_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.locus_batch_plan select *, cast('done' as varchar) as data_copied from hive.historical.full_products_locus_batch_plan where dt = (select max(dt) from hive.historical.full_products_locus_batch_plan where dt >= cast(current_date as varchar))", "insert into hive.products.notification select *, cast('done' as varchar) as data_copied from hive.historical.full_products_notification where dt = (select max(dt) from hive.historical.full_products_notification where dt >= cast(current_date as varchar))", "insert into hive.products.packaging select *, cast('done' as varchar) as data_copied from hive.historical.full_products_packaging where dt = (select max(dt) from hive.historical.full_products_packaging where dt >= cast(current_date as varchar))", "insert into hive.products.packer_table_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_packer_table_mapping where dt = (select max(dt) from hive.historical.full_products_packer_table_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.partner_incentive_sales select *, cast('done' as varchar) as data_copied from hive.historical.full_products_partner_incentive_sales where dt = (select max(dt) from hive.historical.full_products_partner_incentive_sales where dt >= cast(current_date as varchar))", "insert into hive.products.partner_kapture_data select *, cast('done' as varchar) as data_copied from hive.historical.full_products_partner_kapture_data where dt = (select max(dt) from hive.historical.full_products_partner_kapture_data where dt >= cast(current_date as varchar))", "insert into hive.products.price_deal select *, cast('done' as varchar) as data_copied from hive.historical.full_products_price_deal where dt = (select max(dt) from hive.historical.full_products_price_deal where dt >= cast(current_date as varchar))", "insert into hive.products.pricing_zone_dc_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_pricing_zone_dc_mapping where dt = (select max(dt) from hive.historical.full_products_pricing_zone_dc_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.pricing_zone_polygon_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_pricing_zone_polygon_mapping where dt = (select max(dt) from hive.historical.full_products_pricing_zone_polygon_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.product_dimensions select *, cast('done' as varchar) as data_copied from hive.historical.full_products_product_dimensions where dt = (select max(dt) from hive.historical.full_products_product_dimensions where dt >= cast(current_date as varchar))", "insert into hive.products.product_issue select *, cast('done' as varchar) as data_copied from hive.historical.full_products_product_issue where dt = (select max(dt) from hive.historical.full_products_product_issue where dt >= cast(current_date as varchar))", "insert into hive.products.product_ownership select *, cast('done' as varchar) as data_copied from hive.historical.full_products_product_ownership where dt = (select max(dt) from hive.historical.full_products_product_ownership where dt >= cast(current_date as varchar))", "insert into hive.products.product_quality_issue select *, cast('done' as varchar) as data_copied from hive.historical.full_products_product_quality_issue where dt = (select max(dt) from hive.historical.full_products_product_quality_issue where dt >= cast(current_date as varchar))", "insert into hive.products.product_sku_receiving_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_product_sku_receiving_mapping where dt = (select max(dt) from hive.historical.full_products_product_sku_receiving_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.promo_dist_prod_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_promo_dist_prod_mapping where dt = (select max(dt) from hive.historical.full_products_promo_dist_prod_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.purchase_audit select *, cast('done' as varchar) as data_copied from hive.historical.full_products_purchase_audit where dt = (select max(dt) from hive.historical.full_products_purchase_audit where dt >= cast(current_date as varchar))", "insert into hive.products.rack_block select *, cast('done' as varchar) as data_copied from hive.historical.full_products_rack_block where dt = (select max(dt) from hive.historical.full_products_rack_block where dt >= cast(current_date as varchar))", "insert into hive.products.recommend_products select *, cast('done' as varchar) as data_copied from hive.historical.full_products_recommend_products where dt = (select max(dt) from hive.historical.full_products_recommend_products where dt >= cast(current_date as varchar))", "insert into hive.products.retailer_document select *, cast('done' as varchar) as data_copied from hive.historical.full_products_retailer_document where dt = (select max(dt) from hive.historical.full_products_retailer_document where dt >= cast(current_date as varchar))", "insert into hive.products.retailer_early_picking select *, cast('done' as varchar) as data_copied from hive.historical.full_products_retailer_early_picking where dt = (select max(dt) from hive.historical.full_products_retailer_early_picking where dt >= cast(current_date as varchar))", "insert into hive.products.retailer_fixed_price_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_retailer_fixed_price_mapping where dt = (select max(dt) from hive.historical.full_products_retailer_fixed_price_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.retailer_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_retailer_product_mapping where dt = (select max(dt) from hive.historical.full_products_retailer_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.risk_config select *, cast('done' as varchar) as data_copied from hive.historical.full_products_risk_config where dt = (select max(dt) from hive.historical.full_products_risk_config where dt >= cast(current_date as varchar))", "insert into hive.products.sale_audit select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sale_audit where dt = (select max(dt) from hive.historical.full_products_sale_audit where dt >= cast(current_date as varchar))", "insert into hive.products.shift_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_shift_mapping where dt = (select max(dt) from hive.historical.full_products_shift_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.sku_bag_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sku_bag_mapping where dt = (select max(dt) from hive.historical.full_products_sku_bag_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.sku_bunch select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sku_bunch where dt = (select max(dt) from hive.historical.full_products_sku_bunch where dt >= cast(current_date as varchar))", "insert into hive.products.sku_receiving select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sku_receiving where dt = (select max(dt) from hive.historical.full_products_sku_receiving where dt >= cast(current_date as varchar))", "insert into hive.products.sourcing_driver_info select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sourcing_driver_info where dt = (select max(dt) from hive.historical.full_products_sourcing_driver_info where dt >= cast(current_date as varchar))", "insert into hive.products.sub_category_properties select *, cast('done' as varchar) as data_copied from hive.historical.full_products_sub_category_properties where dt = (select max(dt) from hive.historical.full_products_sub_category_properties where dt >= cast(current_date as varchar))", "insert into hive.products.supplier_product_mapping select *, cast('done' as varchar) as data_copied from hive.historical.full_products_supplier_product_mapping where dt = (select max(dt) from hive.historical.full_products_supplier_product_mapping where dt >= cast(current_date as varchar))", "insert into hive.products.wms_zone select *, cast('done' as varchar) as data_copied from hive.historical.full_products_wms_zone where dt = (select max(dt) from hive.historical.full_products_wms_zone where dt >= cast(current_date as varchar))", "insert into hive.products.zone_aisle select *, cast('done' as varchar) as data_copied from hive.historical.full_products_zone_aisle where dt = (select max(dt) from hive.historical.full_products_zone_aisle where dt >= cast(current_date as varchar))", "insert into hive.products.zone_pricing select *, cast('done' as varchar) as data_copied from hive.historical.full_products_zone_pricing where dt = (select max(dt) from hive.historical.full_products_zone_pricing where dt >= cast(current_date as varchar))", "insert into hive.products.zone_subscribable_product select *, cast('done' as varchar) as data_copied from hive.historical.full_products_zone_subscribable_product where dt = (select max(dt) from hive.historical.full_products_zone_subscribable_product where dt >= cast(current_date as varchar))"]
# # call_queries=["CALL system.sync_partition_metadata(schema_name=>'historical', table_name=>'ut_consumer_cn_cart', mode=>'FULL')"]
# call_queries=["CREATE TABLE IF NOT EXISTS hive.consumer.cn_cart(id Integer, frontend_cart_id varchar, consumer_id Integer, agent_id Integer, distributor_id Integer, n_slot_id Integer, entry_time Integer, total_amount Decimal(8,2), discount Decimal(8,2), status Integer, update_time Integer, coupon_code varchar, order_id Integer, sms_sent Integer, dt varchar) WITH(partitioned_by = array['dt'], external_location = 's3a://presto-prod-otipy/otipy-database/live/consumer/cn_cart/', format='ORC')"]
for q in call_queries:
    cur.execute(q)
    rows = cur.fetchall()
    print(rows)

[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]

In [ ]:
CALL system.sync_partition_metadata(schema_name=>'consumer', table_name=>'cn_cart', mode=>'FULL')

In [42]:
import datetime
import calendar